# Installation & Import

In [ ]:
! pip install selenium
! pip install requests beautifulsoup4
! pip install playwright
! pip install ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 5.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2━━━━━━━━━━━ 2/7 [typing_extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [selenium]6/7 [selenium]ions]


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from datetime import datetime
import time
import pandas as pd
import os

In [2]:
from ipynb.fs.full import selection
Bus_natures, Job_natures, emts = selection.return_lists()

# Web-Scraping

In [57]:
for bus in emts:
    print(bus)

ASEAN Internship
Co-op
Government Summer Job (PSSSIP)
Graduate Job
Internship (above minimum wage)
Internship (below minimum wage)
On-Campus Internship
STEM Internship
Unpaid Work
Volunteer Work
UPOP Internship


In [ ]:
#example for selections
BN_selections = ['Internet / Digital / e-Commerce']
JN_selections = []
emt_selections = ['Graduate Job']

In [ ]:
hrefs = []

#Web-scraping
driver = webdriver.Chrome()
driver.get(url='https://career.hkust.edu.hk/web/job.php?keywords=')

#login_info
login_name = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"i0116")))
login_name.send_keys("YOUR EMAIL", Keys.RETURN)
time.sleep(2)
# Enter your ust username and password
driver.find_element(By.NAME,'passwd').send_keys('YOUR Pwd',Keys.RETURN)
button = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "trust-browser-button")))
driver.find_element(By.ID,"trust-browser-button").click()
time.sleep(2)
driver.find_element(By.ID,'chk1').click()
driver.find_element(By.NAME,'declaration').click()
time.sleep(2)
driver.find_element(By.NAME,'declaration').click()
time.sleep(2)

selection_BN = Select(driver.find_element(By.NAME,'BN[]'))
for BN in BN_selections:
    selection_BN.select_by_visible_text(BN)

selection_JN = Select(driver.find_element(By.NAME,'JN[]'))
for JN in JN_selections:
    selection_JN.select_by_visible_text(JN)

selection_emt = Select(driver.find_element(By.NAME,'EMT[]'))
for emt in emt_selections:
    selection_emt.select_by_visible_text(emt)
driver.find_element(By.XPATH,'//*[@id="job_search_form"]/div[2]/div/div[1]/div[9]/button').click()
deadlines = []
today = datetime.today().date()

while True:
    time.sleep(1)
    ddls = driver.find_elements(By.CSS_SELECTOR,'td.detail-text.large-view')
    job_posts = driver.find_elements(By.CSS_SELECTOR, 'a.job-post')
    for idx,ddl in enumerate(ddls):
        if (idx+1)%4 == 0:
            deadlines.append(ddl.text.strip())
    for i in range(min(len(job_posts), len(ddls))):
        ddl = ddls[i]
        job_post = job_posts[i]
        href = job_post.get_attribute('href')
        if href in hrefs:
            pass
        else:
            hrefs.append(href)
    time.sleep(1)

    next_btns = driver.find_elements(By.CLASS_NAME, 'next')
    if next_btns:
        next_btns[0].click()
    else:
        break

print(f"Collected {len(hrefs)} job links")

#Save cookies so that dun need to fill in login_info one by one
cookies = driver.get_cookies()
driver.quit()

Collected 172 job links


In [51]:
companies = []
roles = []
emails = []
for href in hrefs:
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    request = session.get(href)
    if request.status_code == 200:
        soup = BeautifulSoup(request.text,'html.parser')
    find_all_company = find_all = soup.find_all('td',class_='detail-text')
    find = soup.find_all('td',class_='detail-text')[1].find('font')
    companies.append(find_all_company[0].text.strip())
    roles.append(find.text.strip())
    email_tag = soup.select_one('a.red_link[href^="mailto:"]')
    if email_tag:
        email = email_tag.text.strip()
    else:
        email = None
    emails.append(email)


In [53]:
company_data = pd.DataFrame({'Company':companies,'Role':roles,'Email':emails})
if not os.path.exists('company_email.csv'):
    company_data.to_csv('company_email.csv',index=False,encoding='utf-8')
company_data

,Company,Role,Email
0,The Matcher Limited,R&D Software Engineer,recruitment@the-matcher.com
1,"Beijing Science and Technology Co., Three Fast...",Meituan 2025 Campus Recruitment,None
2,"Shenzhen Xunlei Network Technology Co., Ltd (深...",【服务器开发工程师（web3方向）】,campus@xunlei.com
3,"Shenzhen Xunlei Network Technology Co., Ltd (深...",产品经理（web3方向）,campus@xunlei.com
4,OKBL Technology Company Limited,Graduate HIre - Software Engineer,None
...,...,...,...
167,Linjer Limited,Internationalization Manager,None
168,"Inspur Group Co., Ltd. (浪潮集团有限公司)",Core Product R&D; Product R&D; Technical Support,None
169,Alibaba International Digital Commerce Group,Alibaba International 2025 Fall Campus Recruit...,None
170,Aloha Group Limited,Digital Account Executive,info@alohaonline.asia
